# Modeling

First let's open our train dataset.

In [92]:
import pandas as pd

_train_df = pd.read_csv('./train.csv')
_train_df.head()

,loan_id,account_id,loan_date,loan_amount,duration,payments,status,birth_number,district_id,gender,...,trans_type_count_withdrawal,trans_type_count_credit,trans_type_mean_withdrawal,trans_type_mean_credit,trans_type_std_withdrawal,trans_type_std_credit,last_balance_l,max_balance_l,owner_age_at,account_age
0,4959,2,19,0.142398,0.25,0.325934,1,52,1,1,...,32,22,0.592593,0.407407,0.491352,0.491352,1.113161,2.525063,48,10.433333
1,4961,19,209,0.047413,0.00,0.235219,-1,17,21,0,...,34,46,0.425000,0.575000,0.494343,0.494343,2.060494,6.581886,57,12.933333
2,4973,67,210,0.301732,0.25,0.703949,1,48,16,0,...,88,37,0.704000,0.296000,0.456491,0.456491,0.455617,1.855732,51,18.700000
3,4996,132,265,0.156433,0.00,0.752508,1,56,40,0,...,15,16,0.483871,0.516129,0.499740,0.499740,2.670338,3.455298,51,5.966667
4,5002,173,48,0.187112,0.00,0.898079,1,21,66,1,...,18,12,0.600000,0.400000,0.489898,0.489898,0.851487,1.659909,54,6.200000


In [93]:
_train_df['status'].value_counts()

 1    282
-1     46
Name: status, dtype: int64

In [94]:
_train_df.loc[_train_df['status'] == 1, 'status'] = 0
_train_df.loc[_train_df['status'] == -1, 'status'] = 1

In [95]:
_train_df['status'].value_counts()

0    282
1     46
Name: status, dtype: int64

In [96]:
x_train_df = _train_df.drop(['status'], axis=1)
y_train_df = _train_df['status']

In [97]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import make_scorer, precision_score, recall_score, roc_auc_score
from sklearn.model_selection import GridSearchCV
from sklearn.cluster import KMeans
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC

params = {'class_weight': [{0: 1, 1: v} for v in range(1, 7)]}
# params = [
#   {'C': [1, 10, 100, 1000], 'class_weight': [{0: 1, 1: v} for v in range(1, 7)]},
# ]
# params = {'p': [1, 2], 'weights': ['uniform', 'distance']}
# params = {'activation': ['logistic', 'identity', 'relu', 'tanh']}

grid = GridSearchCV(
    estimator=LogisticRegression(max_iter=1000000),
    # estimator=SVC(probability=True),
    param_grid=params,
    scoring={'precision': make_scorer(precision_score), 'recall': make_scorer(recall_score), 'auc': make_scorer(roc_auc_score)},
    refit='auc',
    cv=4,
    n_jobs=-1,
)
grid.fit(x_train_df, y_train_df)

c:\Users\gonca\FEUP-CAC\venv\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of f AND g EVALUATIONS EXCEEDS LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


GridSearchCV(cv=4, estimator=LogisticRegression(max_iter=1000000), n_jobs=-1,
             param_grid={'class_weight': [{0: 1, 1: 1}, {0: 1, 1: 2},
                                          {0: 1, 1: 3}, {0: 1, 1: 4},
                                          {0: 1, 1: 5}, {0: 1, 1: 6}]},
             refit='auc',
             scoring={'auc': make_scorer(roc_auc_score),
                      'precision': make_scorer(precision_score),
                      'recall': make_scorer(recall_score)})

In [98]:
pd.DataFrame(grid.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_class_weight,params,split0_test_precision,split1_test_precision,split2_test_precision,split3_test_precision,...,mean_test_recall,std_test_recall,rank_test_recall,split0_test_auc,split1_test_auc,split2_test_auc,split3_test_auc,mean_test_auc,std_test_auc,rank_test_auc
0,5.388028,1.712818,0.010000,0.002000,"{0: 1, 1: 1}","{'class_weight': {0: 1, 1: 1}}",0.277778,0.500000,0.000000,0.333333,...,0.270833,0.370220,6,0.771447,0.538412,0.485714,0.527381,0.580739,0.111846,5
1,3.638034,0.782061,0.011001,0.001733,"{0: 1, 1: 2}","{'class_weight': {0: 1, 1: 2}}",0.192982,0.200000,0.222222,0.333333,...,0.356061,0.373063,5,0.676056,0.517286,0.533333,0.554762,0.570359,0.062456,6
2,5.463269,1.403655,0.011000,0.001734,"{0: 1, 1: 3}","{'class_weight': {0: 1, 1: 3}}",0.174603,0.166667,0.454545,0.428571,...,0.462121,0.322081,4,0.633803,0.520487,0.665476,0.596429,0.604049,0.054082,3
3,5.429309,1.385860,0.011561,0.002250,"{0: 1, 1: 4}","{'class_weight': {0: 1, 1: 4}}",0.166667,0.187500,0.352941,0.375000,...,0.505682,0.301672,2,0.612676,0.544814,0.671429,0.589286,0.604551,0.045662,2
4,4.278680,1.327888,0.008002,0.002828,"{0: 1, 1: 5}","{'class_weight': {0: 1, 1: 5}}",0.164179,0.166667,0.333333,0.272727,...,0.505682,0.301672,2,0.605634,0.530730,0.664286,0.567857,0.592127,0.049366,4
5,3.929098,0.763483,0.005002,0.001731,"{0: 1, 1: 6}","{'class_weight': {0: 1, 1: 6}}",0.164179,0.208333,0.318182,0.250000,...,0.571970,0.274223,1,0.605634,0.593470,0.684524,0.560714,0.611085,0.045471,1


In [99]:
estimator = grid.best_estimator_
estimator

LogisticRegression(class_weight={0: 1, 1: 6}, max_iter=1000000)

In [100]:
from sklearn.metrics import confusion_matrix
tn, fp, fn, tp = confusion_matrix(y_train_df, estimator.predict(x_train_df)).ravel()
print(f'True positives: {tp}')
print(f'True negatives: {tn}')
print(f'False positives: {fp}')
print(f'False negatives: {fn}')

True positives: 36
True negatives: 208
False positives: 74
False negatives: 10


In [101]:
roc_auc_score(y_train_df, estimator.predict(x_train_df))

0.7600986740672216

In [102]:
_test_df = pd.read_csv('./test.csv')
_test_df.head()

,loan_id,account_id,loan_date,loan_amount,duration,payments,status,birth_number,district_id,gender,...,trans_type_count_withdrawal,trans_type_count_credit,trans_type_mean_withdrawal,trans_type_mean_credit,trans_type_std_withdrawal,trans_type_std_credit,last_balance_l,max_balance_l,owner_age_at,account_age
0,4962,25,136,0.042905,0.00,0.231001,0,202,68,1,...,113,51,0.689024,0.310976,0.462893,0.462893,5.962548,16.751099,35,16.600000
1,4967,37,260,0.534996,1.00,0.520924,0,130,20,1,...,84,32,0.724138,0.275862,0.446948,0.446948,0.328250,1.197016,46,14.066667
2,4968,38,195,0.180285,0.75,0.208516,0,27,19,0,...,32,23,0.581818,0.418182,0.493260,0.493260,1.587986,2.129720,58,8.466667
3,4986,97,81,0.166865,0.00,0.860816,0,46,74,1,...,85,33,0.720339,0.279661,0.448833,0.448833,1.813376,2.437881,55,15.400000
4,4988,103,135,0.444228,0.50,0.735582,0,255,44,1,...,41,42,0.493976,0.506024,0.499964,0.499964,0.438228,1.575078,30,21.200000


In [103]:
x_test_df = _test_df.drop(['status'], axis=1)

In [104]:
results = {
    'Id': x_test_df['loan_id'],
    'Predicted': estimator.predict_proba(x_test_df)[:, 1]
    }

df = pd.DataFrame(results)
df.to_csv('submission.csv', index=False)